In [1090]:
names = {
    1: "Embedding forward",
    2: "LayerNorm 1",
    3: "QKV projection",
    4: "Attention",
    5: "Attention projection",
    6: "Residual 2",
    7: "LayerNorm 2",
    8: "MLP FC",
    9: "GELU",
    10: "MLP projection",
    11: "Residual 3",
    12: "Final LayerNorm",
    13: "Logits",
    14: "Softmax",
    15: "Cross-entropy forward",
    16: "Cross-entropy backward",
    17: "Logits backward (input gradient)",
    18: "Embedding weight gradient",
    19: "Final LayerNorm backward",
    20: "Residual backward (res_3)",
    21: "MLP projection backward input",
    22: "MLP projection backward weight",
    23: "GELU backward", 
    24: "MLP FC backward input",
    25: "MLP FC backward weight",
    26: "LayerNorm 2 backward",
    27: "Residual backward (res_2)",
    28: "Attention projection backward input",
    29: "Attention projection backward weight",
    30: "Attention backward",
    31: "QKV backward input",
    32: "QKV backward weight",
    33: "LayerNorm 1 backward",
    34: "Embedding backward",
    35: "AdamW update"
}

op_kernels = {
    1: "embedding_forward_device",
    2: "layernorm_forward_device",
    3: "mlp_forward_device",
    4: "attention_forward_device",
    5: "mlp_forward_device",
    6: "residual_forward_device",
    7: "layernorm_forward_device",
    8: "mlp_forward_device",
    9: "gelu_forward_device",
    10: "mlp_forward_device",
    11: "residual_forward_device",
    12: "layernorm_forward_device",
    13: "mlp_forward_device",
    14: "softmax_forward_device",
    15: "cross_entropy_forward_device",
    16: "cross_entropy_backward_device",
    17: "mlp_backward_input_device",
    18: "mlp_backward_weight_device",
    19: "layernorm_backward_device",
    20: "residual_backward_device",
    21: "mlp_backward_input_device",
    22: "mlp_backward_weight_device",
    23: "gelu_backward_device",
    24: "mlp_backward_input_device",
    25: "mlp_backward_weight_device",
    26: "layernorm_backward_device",
    27: "residual_backward_device",
    28: "mlp_backward_input_device",
    29: "mlp_backward_weight_device",
    30: "attention_backward_device",
    31: "mlp_backward_input_device",
    32: "mlp_backward_weight_device",
    33: "layernorm_backward_device",
    34: "embedding_backward_device",
    35: "adamw_kernel_device"
}

In [1091]:
NUM_SM = 28
NUM_STEPS = 10
streams = {i:{} for i in range(NUM_SM)}
with open("streams.txt") as timer_f:
    all = timer_f.read()
    for line in all.split('\n')[:-1]:
        # print(line)
        d = ({x.split('=')[0]:int(x.split('=')[1]) for x in line.split(',')})
        # print(d)
        streams[d["sm"]][d["instr"]] = d

timer = {i:[{} for x in range(NUM_SM)] for i in range(NUM_STEPS)}
with open("timer.txt") as timer_f:
    all = timer_f.read()
    for line in all.split('\n')[:-1]:
        # print(line)
        d = ({x.split('=')[0]:int(x.split('=')[1]) for x in line.split(',')})
        for k, v in streams[d["sm"]][d["instr"]].items():
            d[k] = v
        d["name"] = names[d["op"]]
        d["kernelName"] = op_kernels[d["op"]]
        # print(d)
        timer[d["step"]][d["sm"]][d["instr"]] = d


In [1092]:
timer[0][0][123]

{'step': 0,
 'sm': 0,
 'instr': 123,
 'bar_enter': 47291392,
 'bar_exit': 47293440,
 'instr_end': 49072128,
 'spin_wait': 0,
 'exec_time': 1,
 'op': 14,
 'layer': -1,
 'bar_idx': 122,
 'expected': 12568,
 'start_b_x': 121,
 'end_b_x': 129,
 'inc': 1,
 'name': 'Softmax',
 'kernelName': 'softmax_forward_device'}

In [1093]:
from collections import defaultdict
import numpy as np

In [1094]:
for sm_no, sm in enumerate(timer[0]):
    kern_times = defaultdict(list)
    wait_times = defaultdict(list)
    for instr_no, instr in sm.items():
        kern_times[instr["kernelName"]].append(instr["instr_end"] - instr["bar_exit"])
        wait_times[instr["kernelName"]].append(instr["bar_exit"] - instr["bar_enter"])
    count = 0
    wait_count = 0
    for k, times in kern_times.items():
        count += np.sum(times) / (1_000_000) 
        wait_count += np.sum(wait_times[k]) / (1_000_000)
        print(f"{k} \t = \t {np.sum(wait_times[k]) / (1_000_000)}ms | \t{np.array(times) / (1_000_000)} ms")
    print(f"total_time({sm_no})={count}|{wait_count}")

embedding_forward_device 	 = 	 0.0ms | 	[0.31744] ms
layernorm_forward_device 	 = 	 0.031744ms | 	[0.016384 0.016384 0.01536  0.01536  0.016384 0.01536  0.01536  0.01536
 0.016384 0.014336 0.017408 0.01536  0.01536  0.016384 0.016384 0.01536
 0.016384 0.016384 0.01536  0.01536  0.01536  0.017408 0.01536  0.01536
 0.016384] ms
mlp_forward_device 	 = 	 0.139264ms | 	[ 0.545792  0.193536  0.735232  0.756736  0.570368  0.192512  0.760832
  0.755712  0.571392  0.191488  0.734208  0.756736  0.543744  0.190464
  0.734208  0.756736  0.570368  0.192512  0.759808  0.755712  0.571392
  0.164864  0.734208  0.755712  0.54272   0.192512  0.734208  0.754688
  0.570368  0.191488  0.761856  0.754688  0.543744  0.164864  0.734208
  0.755712  0.543744  0.191488  0.761856  0.754688  0.571392  0.191488
  0.760832  0.647168  0.54272   0.192512  0.734208  0.755712 12.6464  ] ms
attention_forward_device 	 = 	 0.167936ms | 	[0.472064 0.477184 0.47616  0.477184 0.478208 0.473088 0.47616  0.475136
 0.474112 0.47

In [1095]:
for sm_no, sm in enumerate(timer[0]):
    wait_times = defaultdict(list)
    for instr_no, instr in sm.items():
        wait_times[instr["kernelName"]].append(instr["bar_exit"] - instr["bar_enter"])
    count = 0
    for k, times in wait_times.items():
        count += np.sum(times) / (1_000_000)
        print(f"{k} \t = \t {np.sum(times) / (1_000_000)} ms")
    print(f"wait_time({sm_no})={count}")

embedding_forward_device 	 = 	 0.0 ms
layernorm_forward_device 	 = 	 0.031744 ms
mlp_forward_device 	 = 	 0.139264 ms
attention_forward_device 	 = 	 0.167936 ms
residual_forward_device 	 = 	 0.16896 ms
gelu_forward_device 	 = 	 0.191488 ms
softmax_forward_device 	 = 	 0.002048 ms
cross_entropy_backward_device 	 = 	 0.159744 ms
mlp_backward_input_device 	 = 	 0.873472 ms
mlp_backward_weight_device 	 = 	 0.605184 ms
layernorm_backward_device 	 = 	 3.10272 ms
residual_backward_device 	 = 	 0.047104 ms
gelu_backward_device 	 = 	 1.202176 ms
attention_backward_device 	 = 	 0.05632 ms
adamw_kernel_device 	 = 	 0.549888 ms
wait_time(0)=7.2980480000000005
embedding_forward_device 	 = 	 0.0 ms
layernorm_forward_device 	 = 	 0.031744 ms
mlp_forward_device 	 = 	 0.151552 ms
attention_forward_device 	 = 	 0.165888 ms
residual_forward_device 	 = 	 0.164864 ms
gelu_forward_device 	 = 	 0.22016 ms
softmax_forward_device 	 = 	 0.002048 ms
cross_entropy_backward_device 	 = 	 0.15872 ms
mlp_backward_inp

In [1096]:

for sm_no, sm in enumerate(timer[0]):
    kern_times = defaultdict(list)
    for instr_no, instr in sm.items():
        kern_times[instr["name"]+":"+instr["kernelName"]].append(instr["instr_end"] - instr["bar_exit"])
    count = 0
    for k, times in kern_times.items():
        count += np.sum(times) / (1_000_000)
        print(f"{k} \t = \t {np.mean(times) / (1_000_000)} ms")
    print(f"total_time({sm_no})={count}")
    break

Embedding forward:embedding_forward_device 	 = 	 0.31744 ms
LayerNorm 1:layernorm_forward_device 	 = 	 0.015957333333333334 ms
QKV projection:mlp_forward_device 	 = 	 0.557312 ms
Attention:attention_forward_device 	 = 	 0.4753066666666667 ms
Attention projection:mlp_forward_device 	 = 	 0.18747733333333333 ms
Residual 2:residual_forward_device 	 = 	 0.010496 ms
LayerNorm 2:layernorm_forward_device 	 = 	 0.015701333333333335 ms
MLP FC:mlp_forward_device 	 = 	 0.745472 ms
GELU:gelu_forward_device 	 = 	 0.022613333333333333 ms
MLP projection:mlp_forward_device 	 = 	 0.7466666666666666 ms
Residual 3:residual_forward_device 	 = 	 0.010410666666666665 ms
Final LayerNorm:layernorm_forward_device 	 = 	 0.016384 ms
Logits:mlp_forward_device 	 = 	 12.6464 ms
Softmax:softmax_forward_device 	 = 	 1.778688 ms
Cross-entropy backward:cross_entropy_backward_device 	 = 	 0.771072 ms
Logits backward (input gradient):mlp_backward_input_device 	 = 	 13.719552 ms
Embedding weight gradient:mlp_backward_weig

In [1097]:

for sm_no, sm in enumerate(timer[0]):
    kern_times = defaultdict(list)
    for instr_no, instr in sm.items():
        kern_times[instr["name"]+":"+instr["kernelName"]].append(instr["instr_end"] - instr["bar_exit"])
    count = 0
    for k, times in kern_times.items():
        count += np.sum(times) / (1_000_000)
        print(f"{k} \t = \t {np.mean(times) / (1_000_000)} ms")
    print(f"total_time({sm_no})={count}")
    break

Embedding forward:embedding_forward_device 	 = 	 0.31744 ms
LayerNorm 1:layernorm_forward_device 	 = 	 0.015957333333333334 ms
QKV projection:mlp_forward_device 	 = 	 0.557312 ms
Attention:attention_forward_device 	 = 	 0.4753066666666667 ms
Attention projection:mlp_forward_device 	 = 	 0.18747733333333333 ms
Residual 2:residual_forward_device 	 = 	 0.010496 ms
LayerNorm 2:layernorm_forward_device 	 = 	 0.015701333333333335 ms
MLP FC:mlp_forward_device 	 = 	 0.745472 ms
GELU:gelu_forward_device 	 = 	 0.022613333333333333 ms
MLP projection:mlp_forward_device 	 = 	 0.7466666666666666 ms
Residual 3:residual_forward_device 	 = 	 0.010410666666666665 ms
Final LayerNorm:layernorm_forward_device 	 = 	 0.016384 ms
Logits:mlp_forward_device 	 = 	 12.6464 ms
Softmax:softmax_forward_device 	 = 	 1.778688 ms
Cross-entropy backward:cross_entropy_backward_device 	 = 	 0.771072 ms
Logits backward (input gradient):mlp_backward_input_device 	 = 	 13.719552 ms
Embedding weight gradient:mlp_backward_weig

In [1098]:
# for sm_no, sm in enumerate(timer[0]):
for instr_no, instr in timer[0][6].items():
    if instr["spin_wait"] > 0:
        print(timer[0][6][instr["instr"]-1])
    # break

In [1099]:
for sm_no, sm in enumerate(timer[0]):
    for instr_no, instr in sm.items():
        if instr["kernelName"] == "mlp_backward_weight_device":
            print(instr)

{'step': 0, 'sm': 0, 'instr': 126, 'bar_enter': 63802368, 'bar_exit': 63803392, 'instr_end': 78231552, 'spin_wait': 0, 'exec_time': 14, 'op': 18, 'layer': -1, 'bar_idx': 126, 'expected': 192, 'start_b_x': 1350, 'end_b_x': 1125, 'inc': 1, 'name': 'Embedding weight gradient', 'kernelName': 'mlp_backward_weight_device'}
{'step': 0, 'sm': 0, 'instr': 130, 'bar_enter': 79376384, 'bar_exit': 79376384, 'instr_end': 80262144, 'spin_wait': 0, 'exec_time': 0, 'op': 22, 'layer': 11, 'bar_idx': 130, 'expected': 768, 'start_b_x': 20, 'end_b_x': 6, 'inc': 1, 'name': 'MLP projection backward weight', 'kernelName': 'mlp_backward_weight_device'}
{'step': 0, 'sm': 0, 'instr': 133, 'bar_enter': 81418240, 'bar_exit': 81432576, 'instr_end': 82380800, 'spin_wait': 0, 'exec_time': 0, 'op': 25, 'layer': 11, 'bar_idx': 133, 'expected': 192, 'start_b_x': 83, 'end_b_x': 69, 'inc': 1, 'name': 'MLP FC backward weight', 'kernelName': 'mlp_backward_weight_device'}
{'step': 0, 'sm': 0, 'instr': 137, 'bar_enter': 8265

In [1100]:
for sm_no, sm in enumerate(timer[0]):
    for instr_no, instr in sm.items():
        if instr["name"] == "Attention backward":
            print(instr)

{'step': 0, 'sm': 0, 'instr': 138, 'bar_enter': 82922496, 'bar_exit': 82931712, 'instr_end': 84455424, 'spin_wait': 0, 'exec_time': 1, 'op': 30, 'layer': 11, 'bar_idx': 138, 'expected': 576, 'start_b_x': 44, 'end_b_x': 8, 'inc': 1, 'name': 'Attention backward', 'kernelName': 'attention_backward_device'}
{'step': 0, 'sm': 0, 'instr': 152, 'bar_enter': 90763264, 'bar_exit': 90763264, 'instr_end': 92318720, 'spin_wait': 0, 'exec_time': 1, 'op': 30, 'layer': 10, 'bar_idx': 152, 'expected': 576, 'start_b_x': 44, 'end_b_x': 9, 'inc': 1, 'name': 'Attention backward', 'kernelName': 'attention_backward_device'}
{'step': 0, 'sm': 0, 'instr': 166, 'bar_enter': 98329600, 'bar_exit': 98338816, 'instr_end': 99823616, 'spin_wait': 0, 'exec_time': 1, 'op': 30, 'layer': 9, 'bar_idx': 166, 'expected': 576, 'start_b_x': 44, 'end_b_x': 8, 'inc': 1, 'name': 'Attention backward', 'kernelName': 'attention_backward_device'}
{'step': 0, 'sm': 0, 'instr': 180, 'bar_enter': 105888768, 'bar_exit': 105889792, 'ins

In [1101]:
(np.arange(65) * 3 ) % 28

array([ 0,  3,  6,  9, 12, 15, 18, 21, 24, 27,  2,  5,  8, 11, 14, 17, 20,
       23, 26,  1,  4,  7, 10, 13, 16, 19, 22, 25,  0,  3,  6,  9, 12, 15,
       18, 21, 24, 27,  2,  5,  8, 11, 14, 17, 20, 23, 26,  1,  4,  7, 10,
       13, 16, 19, 22, 25,  0,  3,  6,  9, 12, 15, 18, 21, 24])

In [1102]:
timer[0][6][207]

{'step': 0,
 'sm': 6,
 'instr': 207,
 'bar_enter': 121025536,
 'bar_exit': 121035776,
 'instr_end': 122591232,
 'spin_wait': 0,
 'exec_time': 1,
 'op': 30,
 'layer': 6,
 'bar_idx': 208,
 'expected': 576,
 'start_b_x': 32,
 'end_b_x': 45,
 'inc': 1,
 'name': 'Attention backward',
 'kernelName': 'attention_backward_device'}

In [1103]:
(4 * 768),(4 * 768) % 109, (4 * 768)//28

(3072, 20, 109)

In [1104]:
for i in range(110):
    for j in range(28):
        print(i * 28 + j, end=", ")
    print()

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 
28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 
56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 
84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 
112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 
140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 
168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 
196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220

In [1105]:
val = [0 for i in range(3800)]
for i in range(110):
    tot = 0
    for j in range(28):
        count = 110
        if j >= 20:
            count -= 1
        # print(i + tot, end=", ")
        val[i + tot] = i * 28 + j
        tot += count
    # print()
print(val)

[0, 28, 56, 84, 112, 140, 168, 196, 224, 252, 280, 308, 336, 364, 392, 420, 448, 476, 504, 532, 560, 588, 616, 644, 672, 700, 728, 756, 784, 812, 840, 868, 896, 924, 952, 980, 1008, 1036, 1064, 1092, 1120, 1148, 1176, 1204, 1232, 1260, 1288, 1316, 1344, 1372, 1400, 1428, 1456, 1484, 1512, 1540, 1568, 1596, 1624, 1652, 1680, 1708, 1736, 1764, 1792, 1820, 1848, 1876, 1904, 1932, 1960, 1988, 2016, 2044, 2072, 2100, 2128, 2156, 2184, 2212, 2240, 2268, 2296, 2324, 2352, 2380, 2408, 2436, 2464, 2492, 2520, 2548, 2576, 2604, 2632, 2660, 2688, 2716, 2744, 2772, 2800, 2828, 2856, 2884, 2912, 2940, 2968, 2996, 3024, 3052, 1, 29, 57, 85, 113, 141, 169, 197, 225, 253, 281, 309, 337, 365, 393, 421, 449, 477, 505, 533, 561, 589, 617, 645, 673, 701, 729, 757, 785, 813, 841, 869, 897, 925, 953, 981, 1009, 1037, 1065, 1093, 1121, 1149, 1177, 1205, 1233, 1261, 1289, 1317, 1345, 1373, 1401, 1429, 1457, 1485, 1513, 1541, 1569, 1597, 1625, 1653, 1681, 1709, 1737, 1765, 1793, 1821, 1849, 1877, 1905, 1933, 1

In [1106]:
import plotly.graph_objects as go
import plotly.express as px


In [1107]:
boxes = [
    {
        "col": 0,
        "y0": 0.0,
        "y1": 1.2,
        "value": 10,
        "info": {"id": 1, "label": "A", "score": 10}
    },
    {
        "col": 0,
        "y0": 1.2,
        "y1": 2.0,
        "value": 40,
        "info": {"id": 2, "label": "B", "score": 40}
    },
    {
        "col": 1,
        "y0": 0.0,
        "y1": 0.6,
        "value": 25,
        "info": {"id": 3, "label": "C", "score": 25}
    },
]

boxes = []
for sm in timer[0]:
    for instr_no, instr in sm.items():
        boxes.append({
            "col" : instr["sm"],
            "y0" : -instr["bar_exit"] / 1_000_000,
            "y1" : -instr["instr_end"] / 1_000_000,
            "value" : instr["op"],
            "info" : instr
        })


In [1108]:
import hashlib
import plotly.express as px

PALETTE = px.colors.qualitative.Alphabet

def hash_color(value):
    h = int(hashlib.md5(str(value).encode()).hexdigest(), 16)
    return PALETTE[h % len(PALETTE)]


In [1109]:
# colorscale = px.colors.sequential.Viridis
# vmin, vmax = 0, 100

# fig = go.Figure()

# for b in boxes:
#     x = b["col"]
#     y0, y1 = b["y0"], b["y1"]
#     i = b["info"]
#     info = f"{i["name"]}<br/>{i["kernelName"]}<br/>exec_time:{i["exec_time"]}ms<br/>spin_wait:{i["spin_wait"]}ms<br/>"

#     fig.add_trace(
#         go.Scatter(
#             x=[x-0.5, x+0.5, x+0.5, x-0.5, x-0.5],
#             y=[y0, y0, y1, y1, y0],
#             fill="toself",
#             mode="lines",
#             line=dict(width=1, color="rgba(0,0,0,0.6)",),
#               # semi-transparent black
#             fillcolor=hash_color(b["value"]),
#             hoveron="fills",   # 🔑 REQUIRED
#             name=f"",
#             # # customdata=[b["info"]],
#             hovertemplate=(
#                 # ""<b>Box info</b><br>"
#                 # "%{customdata}<br>"
#                 # "y: %{y}<extra></extra>""
#                 "TEST %{x}, %{y}"
#             ),
#             showlegend=False,
#             # fill="toself",
#             # mode="lines",
#             # hoveron="fills",     # required
#             # hoverinfo="skip",    # skip point hover
#             # hovertemplate="TEST<extra></extra>",

            
#         )
#     )

# fig.update_layout(
#         height=4000,          # <- make this BIG (2000–10000 px is fine)
#     xaxis=dict(
#         title="Column",
#         tickmode="linear"
#     ),
#     yaxis=dict(
#         title="Y",
#         autorange=True,
#         fixedrange=False   # allow zoom + scroll
#     ),
#     # hoverlabel=dict(
#     #     bgcolor="white",
#     #     font_size=12
#     # ),
#     hovermode="y"

# )

# fig.write_html(
#     "grid_boxes.html",
#     include_plotlyjs="cdn",   # smaller file, loads plotly from CDN
#     full_html=True
# )
# fig.show()


In [1110]:
# import plotly.graph_objects as go

# fig = go.Figure()

# fig.add_trace(
#     go.Scatter(
#         x=[0, 1, 1, 0, 0],
#         y=[0, 0, 1, 1, 0],
#         mode="lines",
#         fill="toself",
#         line=dict(width=1, color="black"),
#         fillcolor="rgba(255,0,0,0.6)",
#         name="hi",
#         hoveron="fills",
#         hovertemplate="HOVER WORKS<extra></extra>",
#     )
# )

# fig.update_layout(
#     hovermode="closest",
#     width=400,
#     height=400,
# )
# fig.write_html(
#     "test.html",
#     include_plotlyjs="cdn",   # smaller file, loads plotly from CDN
#     full_html=True
# )
# fig.show()


In [1111]:
!pip install bokeh

In [1112]:
import colorsys
import numpy as np

def pastel_palette(n, saturation=0.65, lightness=0.75):
    hues = np.linspace(0, 1, n, endpoint=False)
    colors = []
    for h in hues:
        r, g, b = colorsys.hls_to_rgb(h, lightness, saturation)
        colors.append(f"#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}")
    return colors


In [1113]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper
from bokeh.palettes import Turbo256

# boxes = [
#     dict(col=0, y0=0.0, y1=1.2, value=10, info="{'id':1,'label':'A'}"),
#     dict(col=0, y0=1.2, y1=2.0, value=40, info="{'id':2,'label':'B'}"),
#     dict(col=1, y0=0.0, y1=0.6, value=25, info="{'id':3,'label':'C'}"),
# ]

from bokeh.palettes import Turbo256
import numpy as np

# get sorted unique values
unique_vals = sorted({b["value"] for b in boxes})
palette = pastel_palette(len(unique_vals))

value_to_color = dict(zip(unique_vals, palette))



source = ColumnDataSource({
    "x0":   [b["col"]-0.5   for b in boxes],
    "x1":   [b["col"]+0.5   for b in boxes],
    "y0":    [b["y0"]    for b in boxes],
    "y1":    [b["y1"]    for b in boxes],
    "value": [b["value"] for b in boxes],
    "color": [value_to_color[b["value"]] for b in boxes],

    "info":  [f"{b["info"]["name"]}\n{b["info"]["kernelName"]}\nexec_time:{b["info"]["exec_time"]}ms"  for b in boxes],
})

# mapper = LinearColorMapper(
#     palette=Turbo256,
#     low=0,
#     high=100
# )

p = figure(
    width=800,
    height=3000,          # tall → scrolls naturally
    tools="pan,wheel_zoom,reset",
)

p.quad(
    left="x0",
    right="x1",
    bottom="y0",
    top="y1",
    source=source,
    fill_color="color",   # ← direct mapping
    line_color="black",
    line_width=0.5
)

p.add_tools(
    HoverTool(
        tooltips=[
            ("value", "@value"),
            ("info", "@info"),
            ("y-range", "@y0 → @y1"),
        ]
    )
)

# ---------- Y axis ----------
p.yaxis.axis_label =  "time (ms)                   |"
p.yaxis.axis_label_text_font_size = "22pt"
p.yaxis.axis_label_text_font_style = "normal"

p.yaxis.major_label_text_font_size = "16pt"

# Move y-axis title upward
p.yaxis.axis_label_standoff = -50
p.yaxis.axis_label_align = "end"

# ---------- X axis ----------
p.xaxis.major_label_text_font_size = "8pt"


num_cols = 28

p.xaxis.ticker = list(range(num_cols))
p.xaxis.major_label_overrides = {
    i: f"{i}" for i in range(num_cols)
}


# Move x-axis to the top
# move existing x-axis to the top
p.add_layout(p.xaxis[0], 'above')

# remove bottom x-axis
# p.xaxis.visible = False

y_min = min(b["y0"] for b in boxes)

from bokeh.models import Range1d
p.y_range = Range1d(start=y_min, end=0)

from bokeh.models import CustomJSTickFormatter

p.yaxis.formatter = CustomJSTickFormatter(
    code="""
        if (tick <= 0) {
            return Math.abs(tick).toFixed(2);
        } else {
            return "";
        }
    """
)



from bokeh.models import AdaptiveTicker

p.yaxis.ticker = AdaptiveTicker(
    desired_num_ticks=20,   # try 12–20 for tall plots
    min_interval=0,
)


output_file("grid_boxes.html")
save(p)


'/workspace/gpt2.cu/build/grid_boxes.html'